In [2]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)

base_path: /home/miru/sr-press


In [3]:
from express.databases import SQLiteDatabase
from express.datasets import PressingDataset

from express import features as fs
from express import labels as ls

In [4]:
TRAIN_DB_1_PATH = os.path.join(base_path, "stores/train_database (1).sqlite")
TRAIN_DB_PATH = os.path.join(base_path, "stores/train_database.sqlite")

TEST_DB_1_PATH = os.path.join(base_path, "stores/test_database (1).sqlite")
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")

train_db_1= SQLiteDatabase(TRAIN_DB_1_PATH)
train_db = SQLiteDatabase(TRAIN_DB_PATH)

test_db_1 = SQLiteDatabase(TEST_DB_1_PATH)
test_db = SQLiteDatabase(TEST_DB_PATH)

print("train_db_1:", train_db_1)
print("train_db:", train_db)

print("test_db_1:", test_db_1)
print("test_db:", test_db)

train_db_1: <express.databases.sqlite.SQLiteDatabase object at 0x7f8374095460>
train_db: <express.databases.sqlite.SQLiteDatabase object at 0x7f8374095a60>
test_db_1: <express.databases.sqlite.SQLiteDatabase object at 0x7f809a614100>
test_db: <express.databases.sqlite.SQLiteDatabase object at 0x7f836c5e9f70>


In [6]:
print(train_db_1.games().shape, test_db_1.games().shape)

(136, 11) (64, 11)


In [7]:
all_features = [f.__name__ for f in fs.all_features]
all_labels = [f.__name__ for f in ls.all_labels]
print("Features:", all_features)
print("Labels:", all_labels)

Features: ['actiontype', 'actiontype_onehot', 'result', 'result_onehot', 'bodypart', 'bodypart_onehot', 'time', 'startlocation', 'relative_startlocation', 'endlocation', 'relative_endlocation', 'startpolar', 'endpolar', 'movement', 'team', 'time_delta', 'space_delta', 'goalscore', 'angle', 'under_pressure', 'speed', 'freeze_frame_360', 'dist_opponent', 'defenders_in_3m_radius', 'closest_11_players', 'get_column_sum_to_player']
Labels: ['concede_shots', 'counterpress', 'possession_change_by_2_actions', 'possession_change_by_4_actions', 'possession_change_by_6_actions', 'possession_change_by_2_actions_and_3m_distance', 'possession_change_by_4_actions_and_3m_distance', 'possession_change_by_6_actions_and_3m_distance', 'possession_change_by_2_actions_and_5m_distance', 'possession_change_by_4_actions_and_5m_distance', 'possession_change_by_6_actions_and_5m_distance', 'possession_change_by_2_actions_and_7m_distance', 'possession_change_by_4_actions_and_7m_distance', 'possession_change_by_6_a

In [8]:
train_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "train"), 
    xfns=["startlocation", "freeze_frame_360"],
    yfns=["counterpress","possession_change_by_5_seconds"], 
    load_cached =False,
    nb_prev_actions = 3,
)

test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "test"), 
    xfns=["startlocation", "freeze_frame_360"],
    yfns=["counterpress","possession_change_by_5_seconds"], 
    load_cached =False,
    nb_prev_actions = 3,
)

In [9]:
train_dataset.create(train_db_1)
test_dataset.create(test_db_1)

100%|██████████| 64/64 [01:34<00:00,  1.47s/it]


In [13]:
train_dataset.labels['possession_change_by_5_seconds'].value_counts()

possession_change_by_5_seconds
False    26607
True     10341
Name: count, dtype: int64

In [11]:
test_dataset.features

start_x_a0  start_y_a0  start_x_a1  start_y_a1  start_x_a2  \
game_id action_id                                                               
3857254 4             35.6125      44.200     65.0125      23.970     51.4500   
        14            29.7500      28.305     72.1875      42.330     97.3875   
        16            30.8875      13.940     71.3125      41.905     72.1875   
        20            17.1500      60.095     84.7875       9.010     71.4875   
        32            86.0125      58.395     18.6375      17.000     30.1000   
...                       ...         ...         ...         ...         ...   
3869685 2914          62.9125      41.735     42.2625      28.305     19.2500   
        2920          13.6500      52.105     58.0125      15.215     50.3125   
        2931          45.8500      44.625     52.9375      20.995     56.3500   
        2938           3.9375      17.935     99.2250      56.440     99.2250   
        2941           6.3000      21.760     99.7500      52.275     99.7500   

                   start_y_a2  start_x_a3  start_y_a3  \
game_id action_id                                       
3857254 4              33.830     50.3125      37.315   
        14             36.210    104.5625       0.425   
        16             42.330     97.3875      36.210   
        20             54.825     71.3125      55.080   
        32              4.080     56.9625       5.355   
...                       ...         ...         ...   
3869685 2914           39.355     19.4250      34.680   
        2920           15.385     37.0125      22.440   
        2931           18.445     56.3500      18.275   
        2938           56.440    100.8875      61.200   
        2941           52.275     99.2250      56.440   

                                                 freeze_frame_360_a0  \
game_id action_id                                                      
3857254 4          [{'teammate': True, 'actor': False, 'keeper': ...   
        14         [{'teammate': False, 'actor': False, 'keeper':...   
        16         [{'teammate': False, 'actor': False, 'keeper':...   
        20         [{'teammate': True, 'actor': False, 'keeper': ...   
        32         [{'teammate': False, 'actor': False, 'keeper':...   
...                                                              ...   
3869685 2914       [{'teammate': True, 'actor': False, 'keeper': ...   
        2920       [{'teammate': True, 'actor': False, 'keeper': ...   
        2931       [{'teammate': False, 'actor': False, 'keeper':...   
        2938       [{'teammate': True, 'actor': False, 'keeper': ...   
        2941       [{'teammate': True, 'actor': False, 'keeper': ...   

                                                 freeze_frame_360_a1  \
game_id action_id                                                      
3857254 4          [{'teammate': True, 'actor': False, 'keeper': ...   
        14         [{'teammate': True, 'actor': True, 'keeper': F...   
        16         [{'teammate': True, 'actor': True, 'keeper': F...   
        20         [{'teammate': True, 'actor': False, 'keeper': ...   
        32         [{'teammate': True, 'actor': True, 'keeper': F...   
...                                                              ...   
3869685 2914       [{'teammate': True, 'actor': False, 'keeper': ...   
        2920                                                    None   
        2931       [{'teammate': True, 'actor': False, 'keeper': ...   
        2938                                                    None   
        2941       [{'teammate': False, 'actor': False, 'keeper':...   

                                                 freeze_frame_360_a2  \
game_id action_id                                                      
3857254 4          [{'teammate': True, 'actor': False, 'keeper': ...   
        14         [{'teammate': True, 'actor': False, 'keeper': ...   
        16         [{'teammate': True, 'actor': True, 'keeper

In [17]:
train_dataset.labels

,counterpress
0,False
1,False
2,False
3,True
4,True
...,...
39041,False
39042,False
39043,False
39044,False


In [18]:
test_dataset.labels

,counterpress
0,False
1,False
2,False
3,False
4,False
...,...
12401,False
12402,False
12403,False
12404,False


In [19]:
train_dataset.labels["counterpress"].value_counts()

False    31605
True      7441
Name: counterpress, dtype: int64

In [20]:
test_dataset.labels["counterpress"].value_counts()

False    9954
True     2452
Name: counterpress, dtype: int64